In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import pickle
import re
from pymongo import MongoClient
from collections import defaultdict
from nltk.corpus import wordnet as wn
import nltk
from nltk import word_tokenize
from nltk.util import ngrams as ngrams_creator
from nltk.tokenize import word_tokenize
from string import punctuation 
from nltk.corpus import stopwords 
import pickle
nltk.download('vader_lexicon')
from nltk.corpus import sentiwordnet as swn
from nltk.sentiment.vader import SentimentIntensityAnalyzer

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **CSV dataset load**

In [ ]:
# train_path = "/content/drive/MyDrive/NLP/final_processed.csv" #/content/drive/MyDrive/NLP/hindi-english/train_14k_split_no_sw.csv
# test_path = "/content/drive/MyDrive/NLP/val_processed.csv"
train_path = "/data/final_processed.csv" #loading the preprocessed data
test_path = "/data/val_processed.csv" 
df_train = pd.read_csv(train_path)
df_val = pd.read_csv(test_path)

In [ ]:
df_train

,Unnamed: 0,id,sentence,label,sentiment,positive_emoji,negative_emoji,neutral_emoji
0,0,4330,nen á vist bolest vztek smutek zmatek osam ě l...,1,neutral,0,0,0
1,1,41616,Haan yaar neha pensive pensive kab karega woh ...,1,neutral,1896,2412,1218
2,2,6648,television media congress ke liye nhi h Ye toh...,0,negative,0,0,0
3,3,2512,All India nrc lagu kare w Kashmir se dhara 370...,2,positive,0,0,0
4,4,610,Pagal hai kya They real issues Mandir importan...,1,neutral,0,0,0
...,...,...,...,...,...,...,...,...
13960,13960,31686,Saale Gaandu khud porn dikhta hai aur dusre ko...,0,negative,6845,3614,4163
13961,13961,34552,could n't get one cover birthday im going open...,2,positive,0,0,0
13962,13962,16924,Aap logo ki baat nahi kar raha Najim bhai Medi...,2,positive,0,0,0
13963,13963,5556,Jay Jay Shree Ram ram rajye aaya danavo raksha...,2,positive,0,0,0


In [ ]:
df_val

,Unnamed: 0,id,sentence,label,sentiment,positive_emoji,negative_emoji,neutral_emoji
0,0,30258,modi mantrimandal may samil honay par badhai n...,2,positive,0,0,0
1,1,16648,Rashid Tu toh naamakool hai Mare h,0,negative,0,0,0
2,2,28511,U saw caste religion nation saw talent trust p...,0,negative,0,0,0
3,3,10466,sir local police station pe complaint krne par...,1,neutral,0,0,0
4,4,19266,Ve Maahi song Kesari current favourite Music M...,2,positive,0,0,0
...,...,...,...,...,...,...,...,...
2991,2991,16859,Kisi Ko koi Khushi nahi Nayi sarkaar aati naye...,0,negative,0,0,0
2992,2992,2294,Music life Thank Chhote Ustad ( Salman Ali ) p...,1,neutral,0,0,0
2993,2993,29819,gilmour Hmmmm really Sam outlaw NOT something ...,1,neutral,0,0,0
2994,2994,34181,Ab gala faad ke nahi chillana chowkidaar chor ...,0,negative,0,0,0


In [ ]:
data = pd.concat([df_train,df_val]).reset_index()

In [ ]:
data.shape

(16961, 9)

In [ ]:
# trainX = df_train['sentence']
# trainY = df_train["sentiment"]
# df_train

In [ ]:
# data = df_train[["sentence","sentiment","positive_emoji","negative_emoji","neutral_emoji"]]
data.iloc[1].sentence

'Haan yaar neha pensive pensive kab karega woh post sob sob Usne na sach mein photoshoot karna chahiye phir woh post karega'

In [ ]:
data.shape

(16961, 9)

##**preprocess**


In [ ]:

def handle_short_forms(w):
    if w == 'h':
        return 'hai'
    elif w == 'n':
        return 'na'
    elif w == 'da':
        return 'the'
    elif w == 'wid':
        return 'with'
    elif w == 'pr':
        return 'par'
    elif w == 'mattt':
        return 'mat'
    elif w == 'vo':
        return 'woh'
    elif w == 'ki':
        return 'kee'
    elif w == 'ap':
        return 'aap'
    elif w == 'bs':
        return 'bas'
    elif w == 'goood':
        return 'very good'
    elif w == 'tera':
        return 'teraa'
    elif w == 'cnfsn':
        return 'confusion'
    elif w == 'ka':
        return 'kaa'
    elif w == 'rkhi':
        return 'rakhi'
    elif w == 'thts':
        return 'thats'
    elif w == 'cald':
        return 'called'
    elif w == 'tabhe':
        return 'tabhi'
    elif w == 'pta':
        return 'pata'
    elif w == 'b':
        return 'bhi'
    elif w == 'nai':
        return 'nahi'
    elif w == 'f':
        return 'of'
    elif w == 'd':
        return 'the'
    else:
        return w
    
'''
Translate word . 
'''
def translate(word):
    return translator.translate(word,src='hi' , dest='en').text

'''
Self defined contractions
''' 
def load_dict_contractions():
    
    return {
        "ain't":"is not",
        "amn't":"am not",
        "aren't":"are not",
        "can't":"cannot",
        "'cause":"because",
        "couldn't":"could not",
        "couldn't've":"could not have",
        "could've":"could have",
        "daren't":"dare not",
        "daresn't":"dare not",
        "dasn't":"dare not",
        "didn't":"did not",
        "doesn't":"does not",
        "don't":"do not",
        "e'er":"ever",
        "em":"them",
        "everyone's":"everyone is",
        "finna":"fixing to",
        "gimme":"give me",
        "gonna":"going to",
        "gon't":"go not",
        "gotta":"got to",
        "hadn't":"had not",
        "hasn't":"has not",
        "haven't":"have not",
        "he'd":"he would",
        "he'll":"he will",
        "he's":"he is",
        "he've":"he have",
        "how'd":"how would",
        "how'll":"how will",
        "how're":"how are",
        "how's":"how is",
        "I'd":"I would",
        "I'll":"I will",
        "i'll":"I will",
        "I'm":"I am",
        "I'm'a":"I am about to",
        "I'm'o":"I am going to",
        "isn't":"is not",
        "it'd":"it would",
        "it'll":"it will",
        "it's":"it is",
        "I've":"I have",
        "kinda":"kind of",
        "let's":"let us",
        "mayn't":"may not",
        "may've":"may have",
        "mightn't":"might not",
        "might've":"might have",
        "mustn't":"must not",
        "mustn't've":"must not have",
        "must've":"must have",
        "needn't":"need not",
        "ne'er":"never",
        "o'":"of",
        "o'er":"over",
        "ol'":"old",
        "oughtn't":"ought not",
        "shalln't":"shall not",
        "shan't":"shall not",
        "she'd":"she would",
        "she'll":"she will",
        "she's":"she is",
        "shouldn't":"should not",
        "shouldn't've":"should not have",
        "should've":"should have",
        "somebody's":"somebody is",
        "someone's":"someone is",
        "something's":"something is",
        "that'd":"that would",
        "that'll":"that will",
        "that're":"that are",
        "that's":"that is",
        "there'd":"there would",
        "there'll":"there will",
        "there're":"there are",
        "there's":"there is",
        "these're":"these are",
        "they'd":"they would",
        "they'll":"they will",
        "they're":"they are",
        "they've":"they have",
        "this's":"this is",
        "those're":"those are",
        "'tis":"it is",
        "'twas":"it was",
        "wanna":"want to",
        "wasn't":"was not",
        "we'd":"we would",
        "we'd've":"we would have",
        "we'll":"we will",
        "we're":"we are",
        "weren't":"were not",
        "we've":"we have",
        "what'd":"what did",
        "what'll":"what will",
        "what're":"what are",
        "what's":"what is",
        "what've":"what have",
        "when's":"when is",
        "where'd":"where did",
        "where're":"where are",
        "where's":"where is",
        "where've":"where have",
        "which's":"which is",
        "who'd":"who would",
        "who'd've":"who would have",
        "who'll":"who will",
        "who're":"who are",
        "who's":"who is",
        "who've":"who have",
        "why'd":"why did",
        "why're":"why are",
        "why's":"why is",
        "won't":"will not",
        "wouldn't":"would not",
        "would've":"would have",
        "y'all":"you all",
        "you'd":"you would",
        "you'll":"you will",
        "you're":"you are",
        "you've":"you have",
        "Whatcha":"What are you",
        "whatcha":"What are you",
        "luv":"love",
        "sux":"sucks"
        }

'''
Handling short forms and contractions in the sentences 
'''
long_form_dict = load_dict_contractions()
def expand_sent(sentence):
    final_sent =""
    res = " ".join(long_form_dict.get(ele, ele) for ele in sentence.split()) 
    for word in res.split():
        final_sent += (handle_short_forms(word))+ " "
    return final_sent

##work

In [ ]:
data.shape

(16961, 9)

In [ ]:
l=[]
for sent in data.sentence:
  l.append(expand_sent(sent))

print(l[2:5])

['television media congress ke liye nhi hai Ye toh aapko pata chal hi gya hoga Achha hoga kee Congress ke ', 'All India nrc lagu kare w Kashmir se dhara 370ko khatam kare ham Indian ko apse yahi umid hai ', 'Pagal hai kya They real issues Mandir important Hindu khatre mei jo hai ']


##Slang word Processing



In [ ]:
slang_list = pd.read_csv("/data/Hinglish_Profanity_List.csv",sep=",",header=None,encoding= 'unicode_escape')
slang_list.columns=['hindi_word','meaning','rating']

In [ ]:
slang_list

,hindi_word,meaning,rating
0,badir,idiot,1
1,badirchand,idiot,1
2,bakland,idiot,1
3,bhadva,pimp,2
4,bhootnika,son of a witch,3
...,...,...,...
204,vahiyaat,disgusting,4
205,jihadi,terrorist,4
206,atankvadi,terrorist,4
207,atankwadi,terrorist,4


In [ ]:
def find_slang(sentence):
    slang_exists = False
    total_rating = 0
    for word in sentence.split():
        if not slang_list[slang_list['hindi_word']==word].empty:
            try:
                total_rating += slang_list.iloc[slang_list.index[slang_list['hindi_word'] == word].tolist()[0]]['rating']
            except:
                total_rating +=0
            slang_exists = True
    try:
        total_rating = int(total_rating)
    except:
        total_rating = 0
    return slang_exists,total_rating

In [ ]:
data['slang_exists']= data['sentence'].apply(find_slang)

In [ ]:
print(data.shape)
data

(16961, 10)


,index,Unnamed: 0,id,sentence,label,sentiment,positive_emoji,negative_emoji,neutral_emoji,slang_exists
0,0,0,4330,nen á vist bolest vztek smutek zmatek osam ě l...,1,neutral,0,0,0,"(False, 0)"
1,1,1,41616,Haan yaar neha pensive pensive kab karega woh ...,1,neutral,1896,2412,1218,"(False, 0)"
2,2,2,6648,television media congress ke liye nhi h Ye toh...,0,negative,0,0,0,"(False, 0)"
3,3,3,2512,All India nrc lagu kare w Kashmir se dhara 370...,2,positive,0,0,0,"(False, 0)"
4,4,4,610,Pagal hai kya They real issues Mandir importan...,1,neutral,0,0,0,"(False, 0)"
...,...,...,...,...,...,...,...,...,...,...
16956,2991,2991,16859,Kisi Ko koi Khushi nahi Nayi sarkaar aati naye...,0,negative,0,0,0,"(False, 0)"
16957,2992,2992,2294,Music life Thank Chhote Ustad ( Salman Ali ) p...,1,neutral,0,0,0,"(False, 0)"
16958,2993,2993,29819,gilmour Hmmmm really Sam outlaw NOT something ...,1,neutral,0,0,0,"(False, 0)"
16959,2994,2994,34181,Ab gala faad ke nahi chillana chowkidaar chor ...,0,negative,0,0,0,"(False, 0)"


In [ ]:
data[['slang_existance', 'slang_rating']] = pd.DataFrame(data['slang_exists'].tolist(), index=data.index) 
data =data.drop(columns=['slang_exists'])
data

,index,Unnamed: 0,id,sentence,label,sentiment,positive_emoji,negative_emoji,neutral_emoji,slang_existance,slang_rating
0,0,0,4330,nen á vist bolest vztek smutek zmatek osam ě l...,1,neutral,0,0,0,False,0
1,1,1,41616,Haan yaar neha pensive pensive kab karega woh ...,1,neutral,1896,2412,1218,False,0
2,2,2,6648,television media congress ke liye nhi h Ye toh...,0,negative,0,0,0,False,0
3,3,3,2512,All India nrc lagu kare w Kashmir se dhara 370...,2,positive,0,0,0,False,0
4,4,4,610,Pagal hai kya They real issues Mandir importan...,1,neutral,0,0,0,False,0
...,...,...,...,...,...,...,...,...,...,...,...
16956,2991,2991,16859,Kisi Ko koi Khushi nahi Nayi sarkaar aati naye...,0,negative,0,0,0,False,0
16957,2992,2992,2294,Music life Thank Chhote Ustad ( Salman Ali ) p...,1,neutral,0,0,0,False,0
16958,2993,2993,29819,gilmour Hmmmm really Sam outlaw NOT something ...,1,neutral,0,0,0,False,0
16959,2994,2994,34181,Ab gala faad ke nahi chillana chowkidaar chor ...,0,negative,0,0,0,False,0


In [ ]:
data["slang_existance"]==True

0        False
1        False
2        False
3        False
4        False
         ...  
16956    False
16957    False
16958    False
16959    False
16960    False
Name: slang_existance, Length: 16961, dtype: bool

In [ ]:
#Check slang existing values
for i,j,k in zip(data['slang_existance'],data['sentence'],data['slang_rating']):
  if(i==True):
    print(j,"    ",k)

Baih tere itjey Kya jalti hai Pakistan se chutiye khelne de na unhe bc tu Indian bhi nahi h      10
tani Abe haattt tum flop logo ki aukaad hi Kiya h chutiya log ja aapijaan ki chaplusi kar bad      5
Phir se lut chalu ho gaya Bjp walo tumhe kuch chut mil raha hai ya nahi Milna chahiye q ki      8
jan Sharam haya ki qimat lagwai hai liay kute ko ye seat miliy Har haqomat k talway chatny wala hai ye      2
Haramzade hamare tukdo pe palta hai hamein aakh dikhata hai kutte tu pagal ho gaya      2
Jo bjp ke saath nahi woh atankwadi h aour jo hindu RSS ke saath nahi woh deshdrohi hai aaj kal b      4
Tum chutiya hoo gandu cogress harne wali sub ko pata tha itne buti tarah haarnegi nahi pata hai lavde      14
Stuffs Salman continue fuck chindiwaali filmfare So best luck mote bhains      1
Sir ye oonki aadat may shumaar hbycot se media deshwasion ko bhi pata chalegaki gandi naali k      3
Samay Samay ki baat hi rathee ab ModiIsNoDivider timemagazine ispr kch bolne ki himmat bc      10
O kutte

##Emoji Sentiment Analysis


In [ ]:
emoji_list = pd.read_csv('/data/Emoji_Sentiment_Data.csv',sep=",")

In [ ]:
emoji_list

,Emoji,Unicode codepoint,Occurrences,Position,Negative,Neutral,Positive,Unicode name,Unicode block
0,😂,0x1f602,14622,0.805101,3614,4163,6845,FACE WITH TEARS OF JOY,Emoticons
1,❤,0x2764,8050,0.746943,355,1334,6361,HEAVY BLACK HEART,Dingbats
2,♥,0x2665,7144,0.753806,252,1942,4950,BLACK HEART SUIT,Miscellaneous Symbols
3,😍,0x1f60d,6359,0.765292,329,1390,4640,SMILING FACE WITH HEART-SHAPED EYES,Emoticons
4,😭,0x1f62d,5526,0.803352,2412,1218,1896,LOUDLY CRYING FACE,Emoticons
...,...,...,...,...,...,...,...,...,...
964,➛,0x279b,1,0.011628,0,1,0,DRAFTING POINT RIGHTWARDS ARROW,Dingbats
965,♝,0x265d,1,0.280000,0,1,0,BLACK CHESS BISHOP,Miscellaneous Symbols
966,❋,0x274b,1,0.888889,0,1,0,HEAVY EIGHT TEARDROP-SPOKED PROPELLER ASTERISK,Dingbats
967,✆,0x2706,1,0.557252,0,1,0,TELEPHONE LOCATION SIGN,Dingbats


In [ ]:
def find_emoji(sentence):
    positive = 0
    negative = 0
    neutral  = 0
    sentiment =[]
    for word in sentence.split():
        if not emoji_list[emoji_list['Emoji']==word].empty:
            positive += emoji_list.iloc[emoji_list.index[emoji_list['Emoji'] == word].tolist()[0]]['Positive']
            negative += emoji_list.iloc[emoji_list.index[emoji_list['Emoji'] == word].tolist()[0]]['Negative']
            neutral += emoji_list.iloc[emoji_list.index[emoji_list['Emoji'] == word].tolist()[0]]['Neutral']
    return positive,negative,neutral


In [ ]:
data['emoji_sentiment'] = data['sentence'].apply(find_emoji)

In [ ]:
data[['positive_emoji', 'negative_emoji','neutral_emoji']] = pd.DataFrame(data['emoji_sentiment'].tolist(), index=data.index) 

In [ ]:
data =data.drop(columns=['emoji_sentiment'])

#Display random data

In [ ]:
data.tail(50)

,index,Unnamed: 0,id,sentence,label,sentiment,positive_emoji,negative_emoji,neutral_emoji,slang_existance,slang_rating
16911,2946,2946,31223,Sab jhoot hai employees ka khoon choste ho tum...,0,negative,0,0,0,False,0
16912,2947,2947,13583,Omg one best food life ok hand skin tone 3 ski...,1,neutral,0,0,0,False,0
16913,2948,2948,22250,Uttar joota Maar joota Desh ke gaddar ISI ke k...,0,negative,0,0,0,True,5
16914,2949,2949,12006,Media bhed chaal se due rehena achha decision ...,2,positive,0,0,0,False,0
16915,2950,2950,38473,Kutte ki punch hamesha tedi hi rahegi Leave Pa...,0,negative,0,0,0,False,0
16916,2951,2951,15282,Rajdeep Modi ji ki jeet pe tum excuses le aate...,1,neutral,0,0,0,False,0
16917,2952,2952,44663,happy birthday may Allah shower blessings rain...,2,positive,52,5,31,False,0
16918,2953,2953,30659,Sant rampal ke Andhbhakt sleeper cells ki tara...,0,negative,0,0,0,False,0
16919,2954,2954,28868,9pm k slot pe nk ki trp sad joy speak evil joy...,1,neutral,0,0,0,False,0
16920,2955,2955,29767,adarniya ravi bhai sahab aap ko bjp ki jeet pa...,1,neutral,0,0,0,False,0


## Upar wala part karana hai

##Calculating the intensity of sentiment in a sentence using NLTK

##Changes

In [ ]:
data.shape

(16961, 11)

In [ ]:
sid=SentimentIntensityAnalyzer()
neg_score=[]
pos_score=[]
neu_score=[]
for sent in data.sentence:
  ss = sid.polarity_scores(sent)
  neg_score.append(ss['neg'])
  pos_score.append(ss['pos'])
  neu_score.append(ss['neu'])


In [ ]:
neg_score

[0.133,
 0.163,
 0.0,
 0.0,
 0.0,
 0.0,
 0.235,
 0.0,
 0.0,
 0.267,
 0.0,
 0.12,
 0.0,
 0.245,
 0.0,
 0.0,
 0.0,
 0.269,
 0.0,
 0.703,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.186,
 0.242,
 0.0,
 0.0,
 0.167,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.268,
 0.0,
 0.598,
 0.0,
 0.0,
 0.0,
 0.0,
 0.556,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.126,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.194,
 0.202,
 0.0,
 0.0,
 0.0,
 0.13,
 0.0,
 0.0,
 0.115,
 0.0,
 0.0,
 0.194,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.225,
 0.0,
 0.0,
 0.0,
 0.173,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.198,
 0.0,
 0.0,
 0.167,
 0.0,
 0.0,
 0.0,
 0.0,
 0.075,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.106,
 0.0,
 0.35,
 0.0,
 0.224,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.059,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.201,
 0.0,
 0.0,
 0.0,
 0.112,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.15,
 0.37,
 0.0,
 0.403,
 0.204,
 0.363,
 0.0,
 0.0,
 0.

In [ ]:
data['neg_score']= neg_score
data['pos_score']= pos_score
data['neu_score']= neu_score

In [ ]:
data.shape

(16961, 14)

In [ ]:
data.head()

,index,Unnamed: 0,id,sentence,label,sentiment,positive_emoji,negative_emoji,neutral_emoji,slang_existance,slang_rating,neg_score,pos_score,neu_score
0,0,0,4330,nen á vist bolest vztek smutek zmatek osam ě l...,1,neutral,0,0,0,False,0,0.133,0.000,0.867
1,1,1,41616,Haan yaar neha pensive pensive kab karega woh ...,1,neutral,1896,2412,1218,False,0,0.163,0.106,0.732
2,2,2,6648,television media congress ke liye nhi h Ye toh...,0,negative,0,0,0,False,0,0.000,0.000,1.000
3,3,3,2512,All India nrc lagu kare w Kashmir se dhara 370...,2,positive,0,0,0,False,0,0.000,0.000,1.000
4,4,4,610,Pagal hai kya They real issues Mandir importan...,1,neutral,0,0,0,False,0,0.000,0.130,0.870


In [ ]:
data_copy = data
data_copy.shape


(16961, 14)

In [ ]:
data = data[["sentence","sentiment","positive_emoji","negative_emoji","neutral_emoji","slang_existance","slang_rating","neg_score","pos_score","neu_score"]]

In [ ]:
data

,sentence,sentiment,positive_emoji,negative_emoji,neutral_emoji,slang_existance,slang_rating,neg_score,pos_score,neu_score
0,nen á vist bolest vztek smutek zmatek osam ě l...,neutral,0,0,0,False,0,0.133,0.000,0.867
1,Haan yaar neha pensive pensive kab karega woh ...,neutral,1896,2412,1218,False,0,0.163,0.106,0.732
2,television media congress ke liye nhi h Ye toh...,negative,0,0,0,False,0,0.000,0.000,1.000
3,All India nrc lagu kare w Kashmir se dhara 370...,positive,0,0,0,False,0,0.000,0.000,1.000
4,Pagal hai kya They real issues Mandir importan...,neutral,0,0,0,False,0,0.000,0.130,0.870
...,...,...,...,...,...,...,...,...,...,...
16956,Kisi Ko koi Khushi nahi Nayi sarkaar aati naye...,negative,0,0,0,False,0,0.000,0.000,1.000
16957,Music life Thank Chhote Ustad ( Salman Ali ) p...,neutral,0,0,0,False,0,0.000,0.378,0.622
16958,gilmour Hmmmm really Sam outlaw NOT something ...,neutral,0,0,0,False,0,0.133,0.397,0.470
16959,Ab gala faad ke nahi chillana chowkidaar chor ...,negative,0,0,0,False,0,0.000,0.000,1.000


##data Prep

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [ ]:
# removing unwanted patterns from the data
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# Dividing training the test dataset

train = data[:13965]
test = data[13965:]
train.shape ,test.shape 

((13965, 10), (2996, 10))

In [ ]:
##fetching the sentence/comments from the training dataset and cleaning it

train_corpus = []

for i in range(0, len(train)):
  review = re.sub('[^a-zA-Z]', ' ', train['sentence'][i])
  review = review.lower()
  review = review.split()
  
  ps = PorterStemmer()
  
  # stemming
  review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
  
  # joining them back with space
  review = ' '.join(review)
  train_corpus.append(review)

In [ ]:
##fetching the sentence/comments from the test dataset and cleaning it
test_corpus = []

for i in range(len(train), len(data)):
  review = re.sub('[^a-zA-Z]', ' ', test['sentence'][i])
  review = review.lower()
  review = review.split()
  
  ps = PorterStemmer()
  
  # stemming
  review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
  
  # joining them back with space
  review = ' '.join(review)
  test_corpus.append(review)

In [ ]:
#Conversion of snetiment into numerical sentiment

train =train.replace("positive",1)
train =train.replace("negative",2)
train = train.replace("neutral",0)


In [ ]:
test =test.replace("positive",1)
test =test.replace("negative",2)
test = test.replace("neutral",0)

In [ ]:
train

,sentence,sentiment,positive_emoji,negative_emoji,neutral_emoji,slang_existance,slang_rating,neg_score,pos_score,neu_score
0,nen á vist bolest vztek smutek zmatek osam ě l...,0,0,0,0,False,0,0.133,0.000,0.867
1,Haan yaar neha pensive pensive kab karega woh ...,0,1896,2412,1218,False,0,0.163,0.106,0.732
2,television media congress ke liye nhi h Ye toh...,2,0,0,0,False,0,0.000,0.000,1.000
3,All India nrc lagu kare w Kashmir se dhara 370...,1,0,0,0,False,0,0.000,0.000,1.000
4,Pagal hai kya They real issues Mandir importan...,0,0,0,0,False,0,0.000,0.130,0.870
...,...,...,...,...,...,...,...,...,...,...
13960,Saale Gaandu khud porn dikhta hai aur dusre ko...,2,6845,3614,4163,False,0,0.120,0.380,0.500
13961,could n't get one cover birthday im going open...,1,0,0,0,False,0,0.000,0.000,1.000
13962,Aap logo ki baat nahi kar raha Najim bhai Medi...,1,0,0,0,False,0,0.000,0.000,1.000
13963,Jay Jay Shree Ram ram rajye aaya danavo raksha...,1,0,0,0,False,0,0.000,0.000,1.000


In [ ]:
train_corpus

['nen vist bolest vztek smutek zmatek osam lost beznad j nakonec jen klid asi takhl vypad j life',
 'haan yaar neha pensiv pensiv kab karega woh post sob sob usn na sach mein photoshoot karna chahiy phir woh post karega',
 'televis media congress ke liy nhi h ye toh aapko pata chal hi gya hoga achha hoga ki congress ke',
 'india nrc lagu kare w kashmir se dhara ko khatam kare ham indian ko aps yahi umid hai',
 'pagal hai kya real issu mandir import hindu khatr mei jo hai',
 'jeet ki dher sari subh kamnay modi ji asha karta hu jistarah desh ki janta ne kha kar bihar ki jant',
 'topi walay babu ki tau new job hougi humey chutti kesay milay gi sob sob',
 'ye modi media walon ko bhi maza ata hai hindu muslim ke debat karn',
 'baih tere itjey kya jalti hai pakistan se chutiy kheln de na unh bc tu indian bhi nahi h',
 'hehe saw come actual someon shaadi star struck star struck grin face star eye star struck star struck grin face star eye',
 'answer miscalcul vote seat one vote matter deshkam

In [ ]:
train.head()


,sentence,sentiment,positive_emoji,negative_emoji,neutral_emoji,slang_existance,slang_rating,neg_score,pos_score,neu_score
0,nen á vist bolest vztek smutek zmatek osam ě l...,0,0,0,0,False,0,0.133,0.000,0.867
1,Haan yaar neha pensive pensive kab karega woh ...,0,1896,2412,1218,False,0,0.163,0.106,0.732
2,television media congress ke liye nhi h Ye toh...,2,0,0,0,False,0,0.000,0.000,1.000
3,All India nrc lagu kare w Kashmir se dhara 370...,1,0,0,0,False,0,0.000,0.000,1.000
4,Pagal hai kya They real issues Mandir importan...,0,0,0,0,False,0,0.000,0.130,0.870


In [ ]:
'''
bag of words for the training data
'''

bag_of_words_vectorizer = CountVectorizer(max_features=1000)
X = bag_of_words_vectorizer.fit_transform(train_corpus)
words_counts = X.toarray()
tfidf_transformer = TfidfTransformer()
tfidf = tfidf_transformer.fit_transform(words_counts)

y = train.iloc[:, 1] ;y_valid = test.iloc[:, 1]
print(y ,"\n " ,y_valid )

0        0
1        0
2        2
3        1
4        0
        ..
13960    2
13961    1
13962    1
13963    1
13964    0
Name: sentiment, Length: 13965, dtype: int64 
  13965    1
13966    2
13967    2
13968    0
13969    1
        ..
16956    2
16957    0
16958    0
16959    2
16960    0
Name: sentiment, Length: 2996, dtype: int64


In [ ]:
X

<13965x1000 sparse matrix of type '<class 'numpy.int64'>'
	with 116931 stored elements in Compressed Sparse Row format>

In [ ]:
'''
numerical features
'''
numerical_features_t = train[['neg_score','pos_score', 'neu_score', 'positive_emoji', 'negative_emoji', 'neutral_emoji', 'slang_rating']]
print(numerical_features_t.values.tolist())

[[0.133, 0.0, 0.867, 0.0, 0.0, 0.0, 0.0], [0.163, 0.106, 0.732, 1896.0, 2412.0, 1218.0, 0.0], [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.13, 0.87, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0], [0.235, 0.0, 0.765, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 10.0], [0.267, 0.167, 0.567, 0.0, 0.0, 0.0, 0.0], [0.0, 0.084, 0.916, 0.0, 0.0, 0.0, 0.0], [0.12, 0.0, 0.88, 0.0, 0.0, 0.0, 0.0], [0.0, 0.137, 0.863, 0.0, 0.0, 0.0, 0.0], [0.245, 0.254, 0.501, 6361.0, 355.0, 1334.0, 0.0], [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0], [0.269, 0.0, 0.731, 0.0, 0.0, 0.0, 5.0], [0.0, 0.456, 0.544, 0.0, 0.0, 0.0, 0.0], [0.703, 0.0, 0.297, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.13, 0.87, 0.0, 0.0, 0.0, 0.0], [0.0, 0.295, 0.705, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 8.0], [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0], 

In [ ]:
'''
Stacking numerical features along with textual features
'''

bag_of_words_vectorizer.get_feature_names()
combinedFeatures = np.hstack([numerical_features_t, tfidf.toarray()])


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
'''
using TDidf on bag f words
'''

X_test = bag_of_words_vectorizer.transform(test_corpus)
# bag_of_words_vectorizer_test = CountVectorizer(max_features=1000)
# X_test = bag_of_words_vectorizer_test.fit_transform(test_corpus)
words_counts_test = X_test.toarray()
tfidf_test = tfidf_transformer.transform(words_counts_test)

# tfidf_test
# tfidf_transformer_test = TfidfTransformer()
# tfidf_test = tfidf_transformer_test.fit_transform(words_counts_test)

numerical_features_test = test[['neg_score','pos_score', 'neu_score', 'positive_emoji', 'negative_emoji', 'neutral_emoji', 'slang_rating']]
print(numerical_features_test.values.tolist())

[[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0], [0.144, 0.324, 0.532, 0.0, 0.0, 0.0, 0.0], [0.216, 0.0, 0.784, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.57, 0.43, 0.0, 0.0, 0.0, 0.0], [0.0, 0.182, 0.818, 0.0, 0.0, 0.0, 0.0], [0.0, 0.61, 0.39, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.371, 0.629, 0.0, 0.0, 0.0, 0.0], [0.0, 0.416, 0.584, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0], [0.101, 0.523, 0.377, 0.0, 0.0, 0.0, 0.0], [0.138, 0.0, 0.862, 0.0, 0.0, 0.0, 0.0], [0.0, 0.36, 0.64, 108.0, 3.0, 61.0, 0.0], [0.0, 0.245, 0.755, 0.0, 0.0, 0.0, 0.0], [0.0, 0.302, 0.698, 0.0, 0.0, 0.0, 0.0], [0.0, 0.487, 0.513, 0.0, 0.0, 0.0, 0.0], [0.0, 0.368, 0.632, 0.0, 0.0, 0.0, 0.0], [0.0, 0.713, 0.287, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0], [0.189, 0.371

In [ ]:
'''
Stacking numerical features along with textual features
'''

# bag_of_words_vectorizer_test.get_feature_names()
combinedFeatures_test = np.hstack([numerical_features_test, tfidf_test.toarray()])

In [ ]:
combinedFeatures.shape

(13965, 1007)

In [ ]:
combinedFeatures_test.shape

(2996, 1007)

In [ ]:
y.shape

(13965,)

In [ ]:
y_valid.shape

(2996,)

In [ ]:
from sklearn.model_selection import train_test_split
'''
Train test split
'''
# x_train, x_valid, y_train, y_valid = train_test_split(combinedFeatures, y, random_state = 42)
x_train , y_train = combinedFeatures , y
x_valid , y_valid = combinedFeatures_test , y_valid
print(x_train.shape)
print(x_valid.shape)
print(y_train.shape)
print(y_valid.shape)
print(combinedFeatures_test.shape)

(13965, 1007)
(2996, 1007)
(13965,)
(2996,)
(2996, 1007)


In [ ]:
# standardization

from sklearn.preprocessing import StandardScaler

sc = StandardScaler()

x_train = sc.fit_transform(x_train)
x_valid = sc.transform(x_valid)
x_test = sc.transform(combinedFeatures_test)

In [ ]:
# from sklearn import svm
# from sklearn.model_selection import GridSearchCV

# hyperparameters = {'kernel':('rbf','linear','poly'), 'C':[.1, 1, 5, 10], 'degree':[3,5,8]}

# svc = svm.SVC()
# clf = GridSearchCV(svc, hyperparameters, scoring= 'f1')
# clf.fit(np.array(x_train), np.squeeze(y_train))
# print('best parameters are', clf.best_params_)

In [ ]:
x_train[1].shape

(1007,)

#**SVM Classifier**

In [ ]:
# from sklearn import svm
# from sklearn.model_selection import GridSearchCV
# from sklearn import metrics

# hyperparameters = {'kernel':('rbf','linear','poly'), 'C':[.1, 1, 5, 10], 'degree':[3,5,8]}

# svc = svm.SVC()
# clf = GridSearchCV(svc, hyperparameters, scoring= 'f1')
# clf.fit(np.array(x_train), np.squeeze(y_train))
# print('best parameters are', clf.best_params_)
# clf.fit(np.array(x_train), np.squeeze(y_train))

# prediction = clf.predict(x_valid)
# print('Accuracy is', metrics.accuracy_score(y_valid, prediction))
# print('f1 score is', metrics.f1_score(y_valid, prediction))
# print('Rc_auc_score is ', metrics.roc_auc_score(y_valid, prediction)) 

In [ ]:
'''
SVM Classifier
'''

from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score


# model = SVC(kernel="linear", C= 1)
model = SVC(kernel='linear',decision_function_shape='ovr', class_weight='balanced',random_state=0)
# model = LinearSVC(penalty="l2", loss="mse", dual=True, tol=1e-3)
model.fit(x_train, y_train)

y_pred = model.predict(x_valid)

print("Training Accuracy :", model.score(x_train, y_train))
print("Validation Accuracy :", model.score(x_valid, y_valid))

# calculating the f1 score for the validation set
print("f1 score :", f1_score(y_valid, y_pred,average='macro'))



Training Accuracy : 0.6802721088435374
Validation Accuracy : 0.5807743658210948
f1 score : 0.5826224959728284


In [ ]:
#save SVC model 
import joblib
from joblib import dump, load
#dump(model, "/content/drive/MyDrive/NLP/Models/SVM_1.joblib")

In [ ]:
# ##### Loading the model back
# model_trained = load("/content/drive/MyDrive/NLP/Models/SVM_tweet.joblib")
# model_trained.predict(input)

In [ ]:
model_trained = load("/content/drive/MyDrive/NLP/Models/SVM_1007.joblib")



# **Logistic Regression Model**

In [ ]:
'''
Logistic Regression Model
'''

from sklearn.linear_model import LogisticRegression

model_lr = LogisticRegression()
model_lr.fit(x_train, y_train)

y_pred = model_lr.predict(x_valid)

print("Training Accuracy :", model_lr.score(x_train, y_train))
print("Validation Accuracy :", model_lr.score(x_valid, y_valid))

# calculating the f1 score for the validation set
print("f1 score :", f1_score(y_valid, y_pred,average='macro'))

# confusion matrix
cm = confusion_matrix(y_valid, y_pred)
print(cm)

Training Accuracy : 0.67124955245256
Validation Accuracy : 0.5847797062750334
f1 score : 0.5884732503173007
[[563 296 265]
 [255 638  89]
 [266  73 551]]


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [ ]:
#save Logistic Regression model 

# dump(model_lr, "/content/drive/MyDrive/NLP/Models/Logistic_Regression_tweet.joblib")

['/content/drive/MyDrive/NLP/Models/Logistic_Regression_tweet.joblib']

#**Decision Tree Classifier**

In [ ]:
'''
Decision Tree Classifier
'''

from sklearn.tree import DecisionTreeClassifier

model_dt = DecisionTreeClassifier()
model_dt.fit(x_train, y_train)

y_pred = model_dt.predict(x_valid)

print("Training Accuracy :", model_dt.score(x_train, y_train))
print("Validation Accuracy :", model_dt.score(x_valid, y_valid))

# calculating the f1 score for the validation set
print("f1 score :", f1_score(y_valid, y_pred,average='macro'))

# confusion matrix
cm = confusion_matrix(y_valid, y_pred)
print(cm)

Training Accuracy : 0.9897601145721446
Validation Accuracy : 0.5120160213618158
f1 score : 0.5158195001218085
[[544 304 276]
 [349 508 125]
 [300 108 482]]


In [ ]:
# dump(model_dt, "/content/drive/MyDrive/NLP/Models/descision_tree_tweet.joblib")

['/content/drive/MyDrive/NLP/Models/descision_tree_tweet.joblib']

#

# **Random forest classifier**

In [ ]:
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import make_scorer
from sklearn import metrics

scorer = metrics.make_scorer(metrics.f1_score, average = 'weighted')
rfc = RandomForestClassifier()
hyperparameters = {'max_depth':[2,5,10,20],'n_estimators':[10,30,100]}
clf_rfc = GridSearchCV(rfc, hyperparameters, scoring= scorer)
clf_rfc.fit(np.array(x_train), np.squeeze(y_train))
print('best parameters are', clf_rfc.best_params_) 

best parameters are {'max_depth': 20, 'n_estimators': 100}


In [ ]:
# dump(clf_rfc, "/content/drive/MyDrive/NLP/Models/RFC_tweet.joblib")

In [ ]:
#testing
print(x_test[1])
prediction = clf_rfc.predict(x_test)
print(prediction)
print(pd.unique(prediction))
print('Accuracy is', metrics.accuracy_score(y_valid, prediction))
print(metrics.classification_report(y_valid,prediction))

[-0.45585339 -0.78538779  0.94565398 ... -0.04688417 -0.04053524
 -0.05627354]
[1 2 0 ... 0 2 2]
[1 2 0]
Accuracy is 0.5814419225634179
              precision    recall  f1-score   support

           0       0.48      0.61      0.54      1124
           1       0.67      0.64      0.65       982
           2       0.66      0.49      0.56       890

    accuracy                           0.58      2996
   macro avg       0.61      0.58      0.58      2996
weighted avg       0.60      0.58      0.58      2996



In [ ]:
from xgboost import XGBClassifier

scorer = metrics.make_scorer(metrics.f1_score, average = 'weighted')
hyperparameters = {'n_estimators':[10,20,100],'learning_rate':[0.5,.01, 0.001]}
boost = XGBClassifier()
clf_xg = GridSearchCV(boost, hyperparameters, scoring= scorer)
clf_xg.fit(np.array(x_train),np.squeeze(y_train))
print('best parameters are', clf_xg.best_params_)


In [ ]:
# dump(clf_xg.best_params_, "/content/drive/MyDrive/NLP/Models/XGB_tweet.joblib")

In [ ]:
#testing XGB
prediction = clf_xg.predict(np.array(x_test))
print(pd.unique(prediction))
print('Accuracy is', metrics.accuracy_score(y_valid, prediction))

print(metrics.classification_report(y_valid,prediction))

[1 0 2]
Accuracy is 0.46532156368221944
              precision    recall  f1-score   support

           0       0.41      0.57      0.48      1473
           1       0.52      0.50      0.51      1365
           2       0.53      0.28      0.37      1127

    accuracy                           0.47      3965
   macro avg       0.49      0.45      0.45      3965
weighted avg       0.48      0.47      0.46      3965



# **SVM** gridsearch cv

In [ ]:
##GridsearchCV for SVM

hyperparameters = {'kernel':('rbf','linear','poly'), 'C':[.1, 0.5, 1, 5], 'degree':[3,5,8]}
svc = svm.SVC()
grid = GridSearchCV(svc, hyperparameters, scoring= 'f1')
grid.fit(np.array(x_train), np.squeeze(y_train))
print('best parameters are', grid.best_params_)
grid.fit(np.array(x_train), np.squeeze(y_train))

prediction = grid.predict(x_valid)
print('Accuracy is', metrics.accuracy_score(y_valid, prediction))
print('f1 score is', metrics.f1_score(y_valid, prediction))
print('Rc_auc_score is ', metrics.roc_auc_score(y_valid, prediction)) 

In [ ]:
# dump(grid.best_params_, "/content/drive/MyDrive/NLP/Models/SVM_gridsearchcv.joblib")

#**For Demo**

In [ ]:
model_trained = load("/trained_model_weights/bag_of_words/SVM_1007.joblib")

In [ ]:
!pip install emoji

In [ ]:
def clean_tweet(tweet):
    tweet = re.sub('https.*$', '', tweet)  # remove URLs
    tweet = re.sub('RT|cc', '', tweet)  # remove RT and cc
    tweet = re.sub('#\S+', '', tweet)  # remove hashtags
    tweet = re.sub('@ \S+', '', tweet)  # remove mentions
    tweet = re.sub('[%s]' % re.escape("""!"#$%&'()*+-./:;<=>@[\]^_`{|}~"""), ' ', tweet)  # remove punctuations
    tweet = re.sub('\s+', ' ', tweet) 
    tweet = re.sub('[^a-zA-Z]', ' ', tweet) #remove characters other than alphabets
    tweet = tweet.lower()
    #tweet = word_tokenize(tweet) # remove repeated characters (hellooooo into hello)
    # remove extra whitespace
    return tweet

import emoji

def extract_features(tweet):
  # 1. Emoji Analysis
  pos_emoji, neg_emoji, neu_emoji = find_emoji(tweet)
  
  # 2. Slang Existance and slang rating
  slang_exists, slang_rating = find_slang(tweet)

  # 3. Sentiment Analysis
  sid = SentimentIntensityAnalyzer()
  ss = sid.polarity_scores(tweet)
  neg_score = ss['neg']
  pos_score = ss['pos']
  neu_score = ss['neu']

  # 4. Clean data
  tweet0 = emoji.demojize(tweet)
  tweet1 = clean_tweet(tweet0)
  tweet2 = expand_sent(tweet1)
  review = tweet2.split(" ")
  ps = PorterStemmer()
  review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
  tweet3 = ' '.join(review)
  X = bag_of_words_vectorizer.transform([tweet3])
  # print("X shape",X.shape)
  words_counts = X.toarray()
  # tfidf_transformer = TfidfTransformer()
  tfidf = tfidf_transformer.transform(words_counts)

  a=tfidf[-1]
  numerical_features_test = [neg_score, pos_score, neu_score, pos_emoji, neg_emoji, neu_emoji, slang_rating]
  # b=numerical_features_test.toarray()
  numerical_features_test = np.array(numerical_features_test)
  numerical_features_test = numerical_features_test.reshape((1,7))  
  # print(numerical_features_test)
  F = np.hstack([numerical_features_test, a.toarray()])
  return F

In [ ]:
tweet = "this week bohot dhukh kaa hafta hai"
test_f = extract_features(tweet)
y_pred = model_lr.predict(test_f)
y_pred


# test =test.replace("positive",1)
# test =test.replace("negative",2)
# test = test.replace("neutral",0)

array([0])